<p style="text-align:right; font-weight:bold">Мартиросян Елизавета</p>

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.common.keys import Keys

import numpy as np
import urllib.request
import json
import time

API_KEY = 'YlF2rEOkdoIdWhCGHBa30FqlUs143ra9'

# Лабораторная работа #1 <br>

Получаем данные о статьях через api (The New York Times):

In [ ]:
articles = {}
dates = [ (2020, 9), (2020, 10), (2020, 11), (2020, 12), (2021, 1) ]

for date in dates:
    url = 'https://api.nytimes.com/svc/archive/v1/' + str(date[0]) + '/' + str(date[1]) +'.json?api-key='
    with urllib.request.urlopen(url + API_KEY) as data:
        articles[date] = json.loads(data.read().decode('utf-8'))

Имеем всю информацию о статьях - теги, заголовки, ссылки, категории (из ссылок): 

In [ ]:
article = articles[(2020,9)]['response']['docs'][0]
article['headline'], article['web_url'], article['keywords']

({'main': 'Facebook Could Block Sharing of News Stories in Australia',
  'kicker': None,
  'content_kicker': None,
  'print_headline': 'Facebook and Google Chafe As Australia Weighs Curbs',
  'name': None,
  'seo': None,
  'sub': None},
 'https://www.nytimes.com/2020/08/31/technology/facebook-block-news-stories-australia.html',
 [{'name': 'glocations', 'value': 'Australia', 'rank': 1, 'major': 'N'},
  {'name': 'subject',
   'value': 'Computers and the Internet',
   'rank': 2,
   'major': 'N'},
  {'name': 'subject', 'value': 'News and News Media', 'rank': 3, 'major': 'N'},
  {'name': 'subject', 'value': 'Social Media', 'rank': 4, 'major': 'N'},
  {'name': 'subject',
   'value': 'Regulation and Deregulation of Industry',
   'rank': 5,
   'major': 'N'},
  {'name': 'organizations', 'value': 'Facebook Inc', 'rank': 6, 'major': 'N'},
  {'name': 'organizations', 'value': 'Google Inc', 'rank': 7, 'major': 'N'},
  {'name': 'subject',
   'value': 'Rumors and Misinformation',
   'rank': 8,
   'ma

Посчитаем кол-во статьей в каждой категории (для категории us буду брать подкатегории, потому что слишком большая, при этом некоторые статьи не будут обработаны, но это не страшно так как кол-во статей большое; возможно из ссылок будет извлечено не то - имя_статьи.html, даты - тоже не проблема, так как в каждую реальную категорию попадут только те статьи, которые действительно принадлежат этим категориям):

In [ ]:
categories = {}
failed = 0

for date in dates:
    for article in articles[date]['response']['docs']:
        try:
            category = article['web_url'].split('/')[6] 
            if category == 'us':
                category = article['web_url'].split('/')[7]
            if category not in categories:
                categories[category] = []
            categories[category].append(article)
        except:
            failed += 1
print('Кол-во необработанных статей:', failed)

Кол-во необработанных статей: 101


In [ ]:
sorted_categories = sorted(categories.items(), key=lambda x: len(x[1]), reverse=True)
for category in sorted_categories[:10]:
    print(category[0] + ' : ' + str(len(category[1])))

politics : 2105
world : 2081
opinion : 1601
03 : 1537
business : 1294
arts : 1197
sports : 1057
nyregion : 717
books : 694
elections : 594


Выбираем эти 4 категории:

In [ ]:
selected_categories = ['arts', 'sports', 'business', 'politics']

## Скачивание содержания статей через selenium

In [ ]:
downloaded_articles = []

In [ ]:
firefox_capabilities = DesiredCapabilities.FIREFOX
firefox_capabilities['binary'] = '/home/zile/Загрузки/geckodriver'
driver = webdriver.Remote(
   command_executor='http://127.0.0.1:4444/wd/hub',
   desired_capabilities=firefox_capabilities)

driver.get('https://www.nytimes.com')
driver.find_element_by_xpath('//button[@data-testid="login-button"]').click()
driver.find_element_by_id('js-google-oauth-login').click()

time.sleep(30) # чтобы зайти в аккаунт 

for category in selected_categories:
    category_articles = categories[category]
    number_suitable_articles, i = 0, 0
    
    while number_suitable_articles < 1100 and i < len(category_articles):
        driver.get(category_articles[i]['web_url'])
        text_blocks = driver.find_elements_by_class_name('css-axufdj')
        article_body = ''
        for block in text_blocks:
            article_body += block.text + " "
        article = category_articles[i]
        article['category'] = category
        article['text'] = article_body
        if len(article_body) > 500:
            number_suitable_articles += 1
            downloaded_articles.append(article)
        i += 1

Так как при скачивание могли возникнуть какие-либо проблемы, сделаем отбор статей: сравниваем 'word_count' c кол-вом слов в наших скаченных статьях, значение 'word_count' берет в расчет подписи под фотографиями и abstract + кол-во слов в скаченном тексте у нас будет подсчитано грубо, поэтому возьмем только те статьи, для которых эти значения отличны не более чем на 100:

In [ ]:
count_selected_articles = {'arts': 0, 'sports': 0, 'business': 0, 'politics': 0}
selected_downloaded_articles = []

for article in downloaded_articles:
    if abs(article['word_count'] - len(article['text'].split(' '))) <= 100:
        count_selected_articles[article['category']] += 1
        selected_downloaded_articles.append(article)
print(count_selected_articles)

{'arts': 1005, 'sports': 1018, 'business': 1023, 'politics': 1039}


Проверяем корректность скаченного:

In [ ]:
i = np.random.randint(0, 4000)
selected_downloaded_articles[i]['web_url'], selected_downloaded_articles[i]['text']

('https://www.nytimes.com/2020/09/29/arts/music/youngboy-never-broke-again-arrested.html',
 'Last week the rapper YoungBoy Never Broke Again notched his third No. 1 album in a year. This week he is under arrest. YoungBoy, whose real name is Kentrell Gaulden, was arrested on Monday in Baton Rouge, La., his hometown. According to the Baton Rouge Police Department, he was detained on three drug charges and three weapons charges, including possession of a stolen firearm, and felon in possession of a firearm. The rapper was one of 16 people arrested on drug and firearm charges after the police and the FBI “received information from anonymous concerned citizens” about people brandishing guns at an abandoned lot in Baton Rouge, the police said in a news release. According to that announcement, the police seized 14 guns, $79,000 in cash and an unspecified amount of drugs. YoungBoy was processed early Tuesday by the East Baton Rouge Sheriff’s Office and remained in custody as of Tuesday afterno

In [ ]:
i = np.random.randint(0, 4000)
selected_downloaded_articles[i]['web_url'], selected_downloaded_articles[i]['text']

('https://www.nytimes.com/2020/09/14/arts/national-museum-african-american-history-culture-reopening.html',
 'The National Museum of African American History and Culture in Washington will reopen to the public on Friday — one of four Smithsonian Institution museums that are involved in the latest phase of reopening, the Smithsonian announced on Monday. For the museum, which is dedicated to telling the African-American story for all Americans, the reopening comes just a few months after new developments began to unfold in the nation’s history. The museum was closed in March amid by the pandemic, and since then the nation has erupted in social justice protests addressing racism and police violence after George Floyd was killed in police custody in May. The protests will give the museum a new chapter in its narrative. “It is definitely a changed America,” said Lonnie G. Bunch III, the Secretary of the Smithsonian who was the founding director of the museum. “Its role is still the same, wh

In [ ]:
i = np.random.randint(0, 4000)
selected_downloaded_articles[i]['web_url'], selected_downloaded_articles[i]['text']

('https://www.nytimes.com/2020/10/01/business/plenty-of-work-for-homebuilders-but-few-commercial-construction-jobs.html',
 'Home construction workers, whose livelihoods were on the line as the economy collapsed in the spring, are benefiting from a newly resurgent housing market, while builders of commercial and retail properties struggle. About 40 percent of construction workers are in the residential sector, while 60 percent are in the nonresidential field, according to Ken Simonson, chief economist of the Associated General Contractors of America, a trade group. Mr. Simonson’s group represents the nonresidential part of the business, which has been hard hit by empty office buildings and stores. “The outlook is fairly bleak for nonresidential construction,” he said. “New residential construction is terrific, specifically single family homes.” After plunging in the spring, employment in residential construction rose 2.1 percent from June to August, while nonresidential jobs declined 0.

Сохраняем в файл:

In [ ]:
all_articles = []

for downloaded_article in selected_downloaded_articles:
    article_for_save = {}
    article_for_save['article_id'] = downloaded_article['web_url']
    article_for_save['title'] = downloaded_article['headline']['main']
    article_for_save['category'] = downloaded_article['category']
    article_for_save['tags'] = downloaded_article['keywords']
    article_for_save['abstract'] = downloaded_article['abstract']
    article_for_save['text'] = downloaded_article['text']
    all_articles.append(article_for_save)

with open('catalog.json', 'a', encoding='utf8') as outfile:
    json.dump({ 'catalog' : all_articles }, outfile, ensure_ascii=False)

<br/>